Last updated: 30 Sep 2024

In this file, I want to run the same code as before, but with a mantle grid spacing of 20 km. I want to figure out why the memory doesn't get overwhelmed if I split it into sublayers, given that I still have to make the whole array (big cube, 20km spacing) in the beginning. The first cell will be copied from Mantle layering (as far as I remember, this is the last version of the script that produces all the flux and ratio plots thatwe want):



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
print("imports successful")

grid_count_crust = 640 # would need 637.1 for 20 km spacing
#1275 for 5 km spacing
coordinates = np.linspace(-6371, 6371, grid_count_crust)
grid_1d_size_crust = coordinates[1]-coordinates[0]

# Generate the grid coordinates using meshgrid
x_coordinates, y_coordinates, z_coordinates = np.meshgrid(coordinates, coordinates, coordinates)

# Calculate the distance of each point from the origin using vectorized operations
distances_squared = x_coordinates**2 + y_coordinates**2 + z_coordinates**2

# Find indices where distance is less than or equal to the radius squared
crust_indices = np.logical_and(6350**2 < distances_squared, distances_squared <= 6371**2)

# Extract valid coordinates using boolean indexing
crust_grid = np.stack((x_coordinates[crust_indices], y_coordinates[crust_indices], z_coordinates[crust_indices]), axis=-1)

print("crust grid created successfully")

imports successful
crust grid created successfully


The size of distances_squared is determined by the size of the 3D grid generated by np.meshgrid, which in turn is based on the length of the coordinates array.

Here’s how it works:

    coordinates is a 1D array of length grid_count_crust, which is 640 in this case.
    

    np.meshgrid(coordinates, coordinates, coordinates) creates three 3D grids: x_coordinates, y_coordinates, and z_coordinates. Each of these grids has a shape of (640, 640, 640) because the grid spans 640 points along each of the three dimensions (x, y, and z).
    

    distances_squared is calculated by squaring each element of x_coordinates, y_coordinates, and z_coordinates and adding them together. Since each of the coordinate grids has a shape of (640, 640, 640), the resulting distances_squared array will also have the same shape.

Thus, the size (or shape) of distances_squared is (640, 640, 640), which is a 3D array containing the squared distances of all the points in the grid from the origin.

So the x, y and z coordinates contain the whole grid, and now crust_grid is an array that contains triplets (x, y, z) for all the points in the crust. Great! Now we want to select the indices for the other layers. My hope is that if we make them thin enough, everything will work. 

I am doing this manually for now, just to see if it works.

In [2]:
CLM_indices = np.logical_and(6196**2 < distances_squared, distances_squared <= 6350**2)
CLM_grid = np.stack((x_coordinates[CLM_indices], y_coordinates[CLM_indices], z_coordinates[CLM_indices]), axis=-1)

In [3]:
#DM is between 4202 amd 6196
#a's are just limits of the mini layers to constrain the radius

a0 = 4202
a1 = 4300
a2 = 4400
a3 = 4500
a4 = 4600
a5 = 4700
a6 = 4800
a7 = 4900
a8 = 5000
a9 = 5100
a10 = 5200
a11 = 5300
a12 = 5400
a13 = 5500
a14 = 5600
a15 = 5700
a16 = 5800
a17 = 5900
a18 = 6100
a19 = 6196

a_values = np.array([a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19])

# Initialize an empty object array to hold the DM grids
DM_grids = np.empty(len(a_values) - 1, dtype=object)
DM_indices_array = np.empty(len(a_values) - 1, dtype=object)

for i in range(len(a_values) - 1):
    DM_indices = np.logical_and(a_values[i]**2 < distances_squared, distances_squared <= a_values[i + 1]**2)
    try:
        # Attempt to create the DM_grid
        DM_grid = np.stack((x_coordinates[DM_indices], y_coordinates[DM_indices], z_coordinates[DM_indices]), axis=-1)
        
        # Store the DM_grid and DM_indices in the object arrays
        DM_grids[i] = DM_grid
        DM_indices_array[i] = DM_indices
        
        print(f"DM{i+1} grid done, limits: {a_values[i]} , {a_values[i + 1]}")
    
    except MemoryError:
        # Catch memory errors and print a message
        print(f"MemoryError encountered while creating DM{i+1} grid with limits: {a_values[i]} , {a_values[i + 1]}")




DM1 grid done, limits: 4202 , 4300
DM2 grid done, limits: 4300 , 4400
DM3 grid done, limits: 4400 , 4500
DM4 grid done, limits: 4500 , 4600
DM5 grid done, limits: 4600 , 4700
DM6 grid done, limits: 4700 , 4800
DM7 grid done, limits: 4800 , 4900
DM8 grid done, limits: 4900 , 5000
DM9 grid done, limits: 5000 , 5100
DM10 grid done, limits: 5100 , 5200
DM11 grid done, limits: 5200 , 5300
DM12 grid done, limits: 5300 , 5400
DM13 grid done, limits: 5400 , 5500
DM14 grid done, limits: 5500 , 5600
DM15 grid done, limits: 5600 , 5700
DM16 grid done, limits: 5700 , 5800
DM17 grid done, limits: 5800 , 5900
DM18 grid done, limits: 5900 , 6100
DM19 grid done, limits: 6100 , 6196


This worked quite well, let's see if we can create the ones for EM as well. It should be fine, given that DM was the chonkiest one

In [9]:
#DM is between 3480 amd 4202

b0 = 3480
b1 = 3600
b2 = 3700
b3 = 3800
b4 = 3900
b5 = 4000
b6 = 4100
b7 = 4202


b_values = np.array([b0, b1, b2, b3, b4, b5, b6, b7])

# Initialize an empty object array to hold the DM grids
EM_grids = np.empty(len(b_values) - 1, dtype=object)
EM_indices_array = np.empty(len(b_values) - 1, dtype=object)

for i in range(len(b_values) - 1):
    EM_indices = np.logical_and(b_values[i]**2 < distances_squared, distances_squared <= b_values[i + 1]**2)
    try:
        # Attempt to create the DM_grid
        EM_grid = np.stack((x_coordinates[DM_indices], y_coordinates[DM_indices], z_coordinates[DM_indices]), axis=-1)
        
        # Store the DM_grid and DM_indices in the object arrays
        EM_grids[i] = EM_grid
        EM_indices_array[i] = EM_indices
        
        print(f"EM{i+1} grid done, limits: {b_values[i]} , {b_values[i + 1]}")
    
    except MemoryError:
        # Catch memory errors and print a message
        print(f"MemoryError encountered while creating EM{i+1} grid with limits: {b_values[i]} , {b_values[i + 1]}")


EM1 grid done, limits: 3480 , 3600
EM2 grid done, limits: 3600 , 3700
EM3 grid done, limits: 3700 , 3800
EM4 grid done, limits: 3800 , 3900
EM5 grid done, limits: 3900 , 4000
EM6 grid done, limits: 4000 , 4100
EM7 grid done, limits: 4100 , 4202


Ok, we have all the grids! I don't know why this takes up less memory, i even have less memory now because I repartitioned my laptop to get windows as well so I have even less memory available. I will ask chatgpt about this

I didn't make it very clear to me why the difference in memory handling is so significant, I feel like chunking it like this or making one big array should occupy rouglhy the same memory, but anyway. I want to print the sizes of the arrays of arrays 

In [10]:
total_memory = DM_grids.nbytes
print(f"Total memory occupied by DM_grids: {total_memory} bytes")


Total memory occupied by DM_grids: 152 bytes


In [ ]:
# My guess was that DM_grid_big would basically contain the same points as DM_grids, just that in DM_grids are split into smaller grids put together into one array
# I tried running the code before and it didn't finish after like 6 minutes, whereas DM_grids took like 13 seconds
# Weird ! Still can't figure out why

'''
DM_indices_big = np.logical_and(4202**2 < distances_squared, distances_squared <= 6196**2)
DM_grid_big = np.stack((x_coordinates[DM_indices_big], y_coordinates[DM_indices_big], z_coordinates[DM_indices_big]), axis=-1)'''